# Цель исследования ответить на ряд специфических вопросов со стороны стартаперов для увеличения охвата целевой аудитории.

# Подключимся к базе и получим информацию

In [11]:
import pandas as pd
from sqlalchemy import create_engine
import requests

In [12]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# 1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [13]:
query = '''
SELECT *
FROM books
'''
test = pd.io.sql.read_sql(query, con = engine)
print('Количество записей =', len(test))
test.head(3)

Количество записей = 1000


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [15]:
test.describe()

,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000


In [16]:
test.isnull().sum()

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

In [17]:
query = '''
SELECT *
FROM books
WHERE publication_date>'2000-01-01'
'''
test = pd.io.sql.read_sql(query, con = engine)
print('Количество книг =', len(test))


Количество книг = 819


# 2.Для каждой книги посчитаем количество обзоров и среднюю оценку

In [18]:
query = '''
SELECT *
FROM reviews
'''
test = pd.io.sql.read_sql(query, con = engine)
test.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [19]:
query = '''
SELECT *
FROM ratings
'''
test = pd.io.sql.read_sql(query, con = engine)
test.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [20]:
query = '''
SELECT title,
       COUNT(DISTINCT review_id) AS total_rev,
       ROUND(AVG(rating), 1) AS avg_rating
FROM books 
     LEFT JOIN ratings AS rat 
     ON books.book_id = rat.book_id
     LEFT JOIN reviews AS rev 
     ON books.book_id = rev.book_id
GROUP BY books.book_id
ORDER BY total_rev DESC
         
'''
test = pd.io.sql.read_sql(query, con = engine)
test.head()

,title,total_rev,avg_rating
0,Twilight (Twilight #1),7,3.7
1,Water for Elephants,6,4.0
2,The Glass Castle,6,4.2
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,The Curious Incident of the Dog in the Night-Time,6,4.1


Вывод: Книга Twilight (Twilight #1) лидер по числу ревью, но не рейтингу.

# 3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры

Таблица publishers
Содержит данные об издательствах:
publisher_id — идентификатор издательства;
publisher — название издательства;

In [21]:
query = '''
SELECT *
FROM publishers
'''
test = pd.io.sql.read_sql(query, con = engine)
test.head(5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [22]:
query = '''
SELECT publisher,
       COUNT(publisher) as total_books            
FROM books 
LEFT JOIN publishers AS pub 
     ON books.publisher_id = pub.publisher_id
WHERE num_pages>=50
GROUP BY publisher
ORDER BY total_books DESC

'''
books_every = pd.io.sql.read_sql(query, con = engine)
books_every.head(1)

,publisher,total_books
0,Penguin Books,42


# 4. Найдем автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;

In [23]:
query = '''
SELECT *
FROM authors
'''
test = pd.io.sql.read_sql(query, con = engine)
test.head(5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [25]:
query = '''
SELECT author,
      ROUND((AVG(rating)),2) AS middle_rating,
      COUNT(rat.book_id) AS total_books
FROM books as b
     LEFT JOIN authors AS aut 
     ON b.author_id = aut.author_id
     LEFT JOIN ratings AS rat 
     ON b.book_id = rat.book_id
GROUP BY author
HAVING COUNT(rat.rating)>=50
ORDER BY middle_rating DESC

'''
books_every_r = pd.io.sql.read_sql(query, con = engine)
books_every_r.head()

,author,middle_rating,total_books
0,Diana Gabaldon,4.30,50
1,J.K. Rowling/Mary GrandPré,4.29,312
2,Agatha Christie,4.28,53
3,Markus Zusak/Cao Xuân Việt Khương,4.26,53
4,J.R.R. Tolkien,4.24,166


In [26]:
# Сделаем группировку по book_id

In [27]:
query = '''
SELECT b.book_id,
      ROUND((AVG(rating)),2) AS middle_rating,
      COUNT(rat.book_id) AS total_books
FROM books as b
     LEFT JOIN authors AS aut 
     ON b.author_id = aut.author_id
     LEFT JOIN ratings AS rat 
     ON b.book_id = rat.book_id
GROUP BY b.book_id
HAVING COUNT(rat.rating)>=50
ORDER BY middle_rating DESC

'''
books_every_r = pd.io.sql.read_sql(query, con = engine)
books_every_r.head()

,book_id,middle_rating,total_books
0,302,4.41,82
1,722,4.39,74
2,299,4.29,80
3,656,4.26,53
4,300,4.25,73


In [28]:
# Получились другие данные

In [29]:
query = '''
SELECT author                
FROM books as b
     LEFT JOIN authors AS aut 
     ON b.author_id = aut.author_id
     LEFT JOIN ratings AS rat 
     ON b.book_id = rat.book_id  
WHERE b.book_id = 
(
SELECT temp.book_id
FROM
(
SELECT b.book_id,
      ROUND((AVG(rating)),2) AS middle_rating          
FROM books as b
     LEFT JOIN authors AS aut 
     ON b.author_id = aut.author_id
     LEFT JOIN ratings AS rat 
     ON b.book_id = rat.book_id
GROUP BY b.book_id
HAVING COUNT(rat.rating)>=50
ORDER BY middle_rating DESC
LIMIT 1) as temp
)
LIMIT 1

'''
books_every = pd.io.sql.read_sql(query, con = engine)
max_rait = books_every_r['middle_rating'][0]
print(f'Автор книги с наивысшим рейтингом {max_rait} и количеством оценок не менее 50: ', books_every['author'][0])

Автор книги с наивысшим рейтингом 4.41 и количеством оценок не менее 50:  J.K. Rowling/Mary GrandPré


# 5. Посчитайм среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [30]:
query = """
SELECT ROUND((AVG(review_cnt)),2) AS middle_review_from_users
FROM (
       SELECT username, 
              COUNT(review_id) AS review_cnt
       FROM reviews 
       WHERE username IN (
                          SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 50)
       GROUP BY  username
      ) AS temp
"""
books_every = pd.io.sql.read_sql(query, con = engine)
books_every.head()


,middle_review_from_users
0,24.33
